In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np

In [3]:
# пути к тренерировочному датасету
PATH = 'drive/MyDrive/day1'
train_path = f'{PATH}/train_dataset'
st_train_data = f'{train_path}/static_data/train_test_static_data.xlsx'
tm_train = f'{train_path}/tm_data'

In [4]:
import pandas as pd

# Загрузка статичных данных
df = pd.read_excel(st_train_data)

# Просмотр первых строк DataFrame
print(df.head(1))

   ID скважины                              Паспортные данные ЭЦН  \
0        10001  {'name': 'ЭЦНМТ 5А-200DP', 'rate_nom_sm3day': ...   

                                  Характеристики ПЭД  \
0  {"manufacturer": "\u0421\u0426 \u042d\u041f\u0...   

                                       Инклинометрия  \
0  {'measured': [10.0, 20.0, 30.0, 40.0, 50.0, 60...   

                                                  ЭК  \
0  {'bottom_depth': 2806.6, 'sections_info': [{'M...   

                                                 НКТ  Пакер  \
0  {'bottom_depth': 2720.69, 'sections_info': [{'...  False   

   Количество ступеней насоса                                      Сепаратор  \
0                         453  {'k_gas_sep': 0.5, 'sep_name': 'ГСМТ-5А-250'}   

   КПД станции управления  ...  Глубина установки насоса  \
0                    0.97  ...                   2751.61   

   Удельное сопротивление кабеля  Длина кабеля  Относительная плотность газа  \
0                            0.0 

In [ ]:
import pandas as pd
import json
import ast

# Безопасная функция для разбора строковых словарей
def safe_load(val):
    if isinstance(val, str):
        try:
            return json.loads(val)
        except:
            return ast.literal_eval(val)
    return val

# Разворачиваем словарь, но оставляем списки как есть
def expand_dict_column_keep_lists(df, column_name, prefix):
    df[column_name] = df[column_name].apply(safe_load)

    def process_dict(d):
        if not isinstance(d, dict):
            return pd.Series({})  
        return pd.Series({f"{prefix}_{k}": v for k, v in d.items()})

    return df[column_name].apply(process_dict)


# Разворачиваем sections_info с ограничением по числу секций
def expand_section_info_as_lists(column_data, prefix):
    expanded = []
    for row in column_data:
        row = safe_load(row)
        sections = row.get("sections_info", [])
        md_list = []
        d_list = []
        for sec in sections:
            md_list.append(sec.get("MD"))
            d_list.append(sec.get("d"))
        expanded.append({
            f"{prefix}_bottom_depth": row.get("bottom_depth"),
            f"{prefix}_roughness": row.get("roughness", None),
            f"{prefix}_MD_list": md_list,
            f"{prefix}_d_list": d_list
        })
    return pd.DataFrame(expanded)


# Копия исходного датафрейма
df_exp = df.copy()

# Разворачиваем нужные словарные поля
df_ecn = expand_dict_column_keep_lists(df_exp, "Паспортные данные ЭЦН", "ЭЦН")
df_ped = expand_dict_column_keep_lists(df_exp, "Характеристики ПЭД", "ПЭД")
df_sep = expand_dict_column_keep_lists(df_exp, "Сепаратор", "Сепаратор")
df_sht = expand_dict_column_keep_lists(df_exp, "Штуцер", "Штуцер")

# Обрабатываем инклинометрию — только первые значения для наглядности
df_incl = df_exp["Инклинометрия"].apply(safe_load)
df_exp["Инклинометрия_measured_0"] = df_incl.apply(lambda x: x["measured"][0] if "measured" in x and x["measured"] else None)
df_exp["Инклинометрия_absolute_0"] = df_incl.apply(lambda x: x["absolute"][0] if "absolute" in x and x["absolute"] else None)

# Извлекаем из колонок "ЭК" и "НКТ" глубину установки, шероховатость
df_ek = expand_section_info_as_lists(df_exp["ЭК"], "ЭК")
df_nkt = expand_section_info_as_lists(df_exp["НКТ"], "НКТ")


# Финальный датафрейм — объединяем всё
df_flat = pd.concat([
    df_exp.drop(columns=[
        "Паспортные данные ЭЦН", "Характеристики ПЭД", "Сепаратор", "Штуцер",
        "Инклинометрия", "ЭК", "НКТ"
    ]),
    df_ecn,
    df_ped,
    df_sep,
    df_sht,
    df_ek,
    df_nkt
], axis=1)

print(df_flat.head())

print(df_flat.columns.tolist())
print(len(df_flat.columns.tolist()))

   ID скважины  Пакер  Количество ступеней насоса  КПД станции управления  \
0        10001  False                         453                    0.97   
1        10002  False                         387                    0.97   
2        10003  False                         459                    0.97   
3        10004  False                         384                    0.97   
4        10005  False                         461                    0.97   

   КПД трансформатора  Глубина установки насоса  \
0                0.97                   2751.61   
1                0.97                   3095.74   
2                0.97                   3137.14   
3                0.97                   3025.85   
4                0.97                   2368.63   

   Удельное сопротивление кабеля  Длина кабеля  Относительная плотность газа  \
0                       0.000000          2760                           0.7   
1                       3.086687          3230                        

In [ ]:
# Подсчитываем долю пропущенных значений в каждом столбце (в %)
missing = df_flat.isnull().mean().sort_values(ascending=False)
print("Пропущенные данные:\n", missing[missing > 0])


Пропущенные данные:
 Сепаратор_k_gas_sep    0.64
Штуцер_d               0.37
Сепаратор_sep_name     0.13
ПЭД_motor_amp_idle     0.09
ЭЦН_stages_max         0.01
dtype: float64


In [7]:
import numpy as np

def is_list_of_numbers(val):
    return isinstance(val, list) and all(isinstance(x, (int, float, np.number)) for x in val)

# Найдём названия колонок с массивами чисел
list_columns = [col for col in df_flat.columns if df_flat[col].apply(is_list_of_numbers).any()]

# Выводим .head() только по этим колонкам
print("📌 Предпросмотр значений с массивами чисел:\n")
print(df_flat[list_columns].head())


📌 Предпросмотр значений с массивами чисел:

                                     ЭЦН_rate_points  \
0  [10.865, 21.729, 32.594, 43.458, 54.323, 65.18...   
1           [0.0, 100.0, 150.0, 200.0, 250.0, 300.0]   
2  [0.0, 50.0, 75.0, 100.0, 125.0, 150.0, 160.0, ...   
3             [0.0, 40.0, 70.0, 100.0, 140.0, 160.0]   
4  [0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 53.0, 60.0...   

                                     ЭЦН_head_points  \
0  [7.107, 7.073, 7.033, 6.993, 6.957, 6.929, 6.9...   
1               [6.02, 5.95, 5.69, 4.82, 3.43, 1.77]   
2  [6.4, 6.5, 6.55, 6.55, 6.4, 5.9, 5.6, 5.0, 3.7...   
3               [8.39, 8.68, 8.35, 7.41, 5.08, 3.24]   
4  [7.17, 7.09, 7.02, 6.99, 6.95, 6.88, 6.83, 6.7...   

                                    ЭЦН_power_points  \
0  [0.11800000000000001, 0.129, 0.139, 0.148, 0.1...   
1  [0.0951, 0.1472, 0.17400000000000002, 0.1832, ...   
2  [0.09870000000000001, 0.127, 0.1386, 0.1523000...   
3   [0.1061, 0.1217, 0.1326, 0.1451, 0.1655, 0.1769]   
4 

In [ ]:
from numpy import trapz  
# Функция для расчёта площади под кривыми (head, power, eff) по данным ЭЦН
def nrx_area_features(df, x_col, y_cols):
    area_data = {}

    # Подготовка имён для новых столбцов с расчётными площадями
    for y in y_cols:
        base = y.replace("ЭЦН_", "").replace("_points", "")
        area_data[f"НРХ_{base}_area"] = []

    # Расчёт площади под кривой для каждой строки DataFrame
    for _, row in df.iterrows():
        x = row.get(x_col)

        for y in y_cols:
            base = y.replace("ЭЦН_", "").replace("_points", "")
            y_vals = row.get(y)

            if isinstance(x, list) and isinstance(y_vals, list) and len(x) == len(y_vals) and len(x) >= 2:
                area = trapz(y_vals, x) 
            else:
                area = np.nan

            area_data[f"НРХ_{base}_area"].append(area)

    # Возвращаем датафрейм с новыми признаками
    return pd.DataFrame(area_data)
df_area = nrx_area_features(
    df=df_flat,
    x_col="ЭЦН_rate_points",
    y_cols=["ЭЦН_head_points", "ЭЦН_power_points", "ЭЦН_eff_points"]
)

# Объединяем с основным датафреймом
df_flat = pd.concat([df_flat.drop(columns=["ЭЦН_rate_points"]), df_area], axis=1)

# Проверяем результат
print(df_flat.filter(like="НРХ_").head())

   НРХ_head_area  НРХ_power_area  НРХ_eff_area
0    1800.417551       64.394247    139.055892
1    1488.500000       47.752500    126.750000
2    1299.750000       35.936000     97.627250
3    1166.250000       22.172000     64.640000
4     815.740000       14.415000     59.756550


<ipython-input-8-b345c6514440>:18: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = trapz(y_vals, x)  # площадь под кривой


In [ ]:
from numpy import trapz

# Функция для расчёта площадей под кривыми ПЭД (по точкам: ток, cosφ, КПД, обороты)
def motor_area_features(df, x_col, y_cols):
    area_data = {}

    # Подготавливаем имена новых признаков
    for y in y_cols:
        base = y.replace("ПЭД_", "").replace("_points", "")
        area_data[f"ПЭД_{base}_area"] = []

    # Проходим по строкам и рассчитываем площадь под каждой кривой
    for _, row in df.iterrows():
        x = row.get(x_col)

        for y in y_cols:
            base = y.replace("ПЭД_", "").replace("_points", "")
            y_vals = row.get(y)

            if isinstance(x, list) and isinstance(y_vals, list) and len(x) == len(y_vals) and len(x) >= 2:
                area = trapz(y_vals, x)
            else:
                area = np.nan

            area_data[f"ПЭД_{base}_area"].append(area)

    return pd.DataFrame(area_data)

# Применяем функцию для расчёта площадей по параметрам ПЭД
df_motor_area = motor_area_features(
    df=df_flat,
    x_col="ПЭД_load_points",
    y_cols=[
        "ПЭД_amperage_points",
        "ПЭД_cosf_points",
        "ПЭД_eff_points",
        "ПЭД_rpm_points"
    ]
)

# Объединяем полученные признаки с основным датафреймом, удаляя исходные точки загрузки ПЭД
df_flat = pd.concat([df_flat.drop(columns=["ПЭД_load_points"]), df_motor_area], axis=1)

print(df_flat.filter(like="ПЭД_").head())


  ПЭД_manufacturer                ПЭД_name  ПЭД_d_motor_mm  ПЭД_motor_nom_i  \
0           СЦ ЭПУ          ПЭДМТ160-117М1           117.0             59.0   
1   Лысьванефтемаш          1ЭДБ140-117 В5           117.0             57.0   
2            Борец  9ЭДБТ100-117Э/2700М8В5           117.0             30.2   
3          Новомет     ПЭДН80-117-2100/041           117.0             30.7   
4     Schlumberger             ПЭДМТ56-103           103.0             35.3   

   ПЭД_motor_amp_idle  ПЭД_motor_nom_power  ПЭД_motor_nom_voltage  \
0                59.0           160.000000                 2250.0   
1                57.0           140.248269                 2000.0   
2                30.2           100.000000                 2700.0   
3                30.7            80.000000                 2100.0   
4                35.3            56.000000                 1400.0   

   ПЭД_motor_nom_eff  ПЭД_motor_nom_cosf  ПЭД_motor_nom_freq  \
0             0.8400               0.840      

<ipython-input-9-c5ba41b69c5e>:18: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = trapz(y_vals, x)


In [ ]:
# Список колонок со списками значений по ЭК и НКТ
cols_to_avg = ["ЭК_MD_list", "ЭК_d_list", "НКТ_MD_list", "НКТ_d_list"]

# Вычисляем среднее значение в каждом списке и сохраняем как новый признак
for col in cols_to_avg:
    df_flat[f"{col}_mean"] = df_flat[col].apply(
        lambda x: sum(x)/len(x) if isinstance(x, list) and len(x) > 0 else np.nan
    )

# Теперь можем удалить оригинальные массивы
df_flat.drop(columns=cols_to_avg, inplace=True)

print(df_flat[[col + "_mean" for col in cols_to_avg]].head())


   ЭК_MD_list_mean  ЭК_d_list_mean  НКТ_MD_list_mean  НКТ_d_list_mean
0      1787.066667      147.933333       1924.275000            59.00
1      2501.425000      146.850000       2034.012500            59.00
2      2951.233333      142.600000       1837.572500            59.00
3      2313.896667      139.466667       1752.256667            58.00
4      1788.200000      117.200000       1540.072500            58.75


In [ ]:
# Выводим значения столбца "ЭЦН_freq_Hz"
a = df_flat.columns.to_list()
for i in a:
  print(i)
print(df_flat["ЭЦН_freq_Hz"])

ID скважины
Пакер
Количество ступеней насоса
КПД станции управления
КПД трансформатора
Глубина установки насоса
Удельное сопротивление кабеля
Длина кабеля
Относительная плотность газа
Относительная плотность нефти
Относительная плотность воды
Пластовое давление
Пластовая температура
Линейная температура
Инклинометрия_measured_0
Инклинометрия_absolute_0
ЭЦН_name
ЭЦН_rate_nom_sm3day
ЭЦН_rate_opt_min_sm3day
ЭЦН_rate_opt_max_sm3day
ЭЦН_freq_Hz
ЭЦН_head_points
ЭЦН_power_points
ЭЦН_eff_points
ЭЦН_stages_max
ЭЦН_rate_max_sm3day
ЭЦН_slip_nom_rpm
ЭЦН_eff_max
ПЭД_manufacturer
ПЭД_name
ПЭД_d_motor_mm
ПЭД_motor_nom_i
ПЭД_motor_amp_idle
ПЭД_motor_nom_power
ПЭД_motor_nom_voltage
ПЭД_motor_nom_eff
ПЭД_motor_nom_cosf
ПЭД_motor_nom_freq
ПЭД_amperage_points
ПЭД_cosf_points
ПЭД_eff_points
ПЭД_rpm_points
Сепаратор_k_gas_sep
Сепаратор_sep_name
Штуцер_d
ЭК_bottom_depth
ЭК_roughness
НКТ_bottom_depth
НКТ_roughness
НРХ_head_area
НРХ_power_area
НРХ_eff_area
ПЭД_amperage_area
ПЭД_cosf_area
ПЭД_eff_area
ПЭД_rpm_a

In [ ]:
import pandas as pd
import os

# Путь к папке
folder_path = tm_train

all_dfs = []
i = 0

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        path = os.path.join(folder_path, filename)
        df = pd.read_csv(path)

        # Оставляем только строки с параметром "дебит жидкости" (param_id == 1)
        df = df[df["param_id"] == 1]

        if not df.empty:
            i += 1
            print(f"Обработан файл №{i}: {filename}")
            all_dfs.append(df[["well_id", "tm_value"]])

# Объединяем всё
df_debit = pd.concat(all_dfs, ignore_index=True)
count_not_nan = df_debit['tm_value'].notna().sum()
print(f"Количество строк с ненулевым дебитом жидкости: {count_not_nan}")

# Группировка и расчёт среднего дебита
df_debit_avg = df_debit.groupby("well_id", as_index=False)["tm_value"].mean()
df_debit_avg = df_debit_avg.rename(columns={"tm_value": "средний дебит"})

print(df_debit_avg.head())


Обработан файл №1: train_10001.csv
Обработан файл №2: train_10002.csv
Обработан файл №3: train_10003.csv
Обработан файл №4: train_10004.csv
Обработан файл №5: train_10005.csv
Обработан файл №6: train_20001.csv
Обработан файл №7: train_20002.csv
Обработан файл №8: train_20003.csv
Обработан файл №9: train_20004.csv
Обработан файл №10: train_20005.csv
Обработан файл №11: train_20006.csv
Обработан файл №12: train_20007.csv
Обработан файл №13: train_20008.csv
Обработан файл №14: train_20009.csv
Обработан файл №15: train_20010.csv
Обработан файл №16: train_20011.csv
Обработан файл №17: train_20012.csv
Обработан файл №18: train_20013.csv
Обработан файл №19: train_20014.csv
Обработан файл №20: train_20015.csv
Обработан файл №21: train_20016.csv
Обработан файл №22: train_20017.csv
Обработан файл №23: train_20018.csv
Обработан файл №24: train_20019.csv
Обработан файл №25: train_20020.csv
Обработан файл №26: train_20021.csv
Обработан файл №27: train_20022.csv
Обработан файл №28: train_20023.csv
О

In [ ]:
df_flat_renamed = df_flat.rename(columns={"ID скважины": "well_id"})

# Объединяем паспортные данные с рассчитанным средним дебитом по скважинам
df_merged = df_flat_renamed.merge(
    df_debit_avg[["well_id", "средний дебит"]],
    on="well_id",
    how="left"
)

print(df_merged.head())


   well_id  Пакер  Количество ступеней насоса  КПД станции управления  \
0    10001  False                         453                    0.97   
1    10002  False                         387                    0.97   
2    10003  False                         459                    0.97   
3    10004  False                         384                    0.97   
4    10005  False                         461                    0.97   

   КПД трансформатора  Глубина установки насоса  \
0                0.97                   2751.61   
1                0.97                   3095.74   
2                0.97                   3137.14   
3                0.97                   3025.85   
4                0.97                   2368.63   

   Удельное сопротивление кабеля  Длина кабеля  Относительная плотность газа  \
0                       0.000000          2760                           0.7   
1                       3.086687          3230                           0.7   
2             

In [14]:
# Только числовые переменные
num_df = df_merged.select_dtypes(include='number')

# Считаем корреляцию со средним дебитом
target = "средний дебит"
corr_with_target = num_df.corr()[target].sort_values(ascending=False)

print("📊 Корреляция признаков со средним дебитом:")
print(corr_with_target)

📊 Корреляция признаков со средним дебитом:
средний дебит                    1.000000
ЭЦН_rate_nom_sm3day              0.421106
Относительная плотность воды     0.421007
ЭЦН_rate_opt_max_sm3day          0.420791
НРХ_eff_area                     0.413380
ЭЦН_rate_opt_min_sm3day          0.400380
ЭЦН_rate_max_sm3day              0.396474
ЭЦН_eff_max                      0.386852
ПЭД_motor_nom_power              0.364411
ПЭД_motor_nom_i                  0.357151
ПЭД_motor_amp_idle               0.355476
НРХ_head_area                    0.354367
НРХ_power_area                   0.335991
Удельное сопротивление кабеля    0.153150
ПЭД_amperage_area                0.111487
ПЭД_motor_nom_voltage            0.110328
Сепаратор_k_gas_sep              0.103382
ЭЦН_slip_nom_rpm                 0.094226
Пластовое давление               0.079227
ПЭД_rpm_area                     0.056350
Инклинометрия_measured_0         0.053336
Инклинометрия_absolute_0         0.053336
ПЭД_eff_area                     

In [ ]:
# Убираем слабые (по модулю) корреляции
threshold = 0.05
useful_features = corr_with_target[abs(corr_with_target) >= threshold].index.tolist()

# Исключаем сам таргет из признаков
useful_features = [f for f in useful_features if f != target]

# Гарантируем, что нужные признаки останутся
must_keep = ['ЭЦН_rate_opt_min_sm3day', 'ЭЦН_rate_opt_max_sm3day']
for feature in must_keep:
    if feature not in useful_features:
        useful_features.append(feature)

# Оставляем только нужные признаки
df_filtered = df_merged[useful_features + [target]]

print("\n Отфильтрованный датафрейм (наиболее полезные признаки):")
print(len(df_filtered.columns.to_list()))



✅ Отфильтрованный датафрейм (наиболее полезные признаки):
37


In [16]:
df_filtered.to_excel("df_filtered.xlsx")

In [17]:
print(df_filtered.columns.to_list())

['ЭЦН_rate_nom_sm3day', 'Относительная плотность воды', 'ЭЦН_rate_opt_max_sm3day', 'НРХ_eff_area', 'ЭЦН_rate_opt_min_sm3day', 'ЭЦН_rate_max_sm3day', 'ЭЦН_eff_max', 'ПЭД_motor_nom_power', 'ПЭД_motor_nom_i', 'ПЭД_motor_amp_idle', 'НРХ_head_area', 'НРХ_power_area', 'Удельное сопротивление кабеля', 'ПЭД_amperage_area', 'ПЭД_motor_nom_voltage', 'Сепаратор_k_gas_sep', 'ЭЦН_slip_nom_rpm', 'Пластовое давление', 'ПЭД_rpm_area', 'Инклинометрия_measured_0', 'Инклинометрия_absolute_0', 'ЭЦН_stages_max', 'НКТ_d_list_mean', 'Длина кабеля', 'ПЭД_motor_nom_freq', 'Глубина установки насоса', 'НКТ_bottom_depth', 'ЭК_MD_list_mean', 'Относительная плотность газа', 'ПЭД_motor_nom_eff', 'Штуцер_d', 'Пластовая температура', 'Линейная температура', 'ЭК_d_list_mean', 'Относительная плотность нефти', 'well_id', 'средний дебит']


In [ ]:
import pandas as pd
import os

# Названия по param_id
id_to_param_name = {
    1: "Дебит жидкости (объёмный), м3/сут",
    2: "Обводненность (объёмная), %",
    3: "Рбуф, атм",
    4: "Давление линейное, атм",
    5: "Давление на приеме насоса, атм",
    6: "Загрузка ПЭД, %",
    7: "Попутный газ",
    8: "Частота тока, Герц",
    9: "Ток фазы А, A (ампер)",
    10: "Мощность активная, кВт",
    11: "Напряжение, АВ Вольт",
    12: "P затрубное, атм",
}

# Диапазоны допустимых значений
value_ranges = {
    "Дебит жидкости (объёмный), м3/сут": [0, 5000],
    "Обводненность (объёмная), %": [0, 100],
    "Рбуф, атм": [1, 300],
    "Давление линейное, атм": [1, 300],
    "Давление на приеме насоса, атм": [1, 300],
    "Загрузка ПЭД, %": [0, 100],
    "Попутный газ": [0, 100000],
    "Частота тока, Герц": [0, 500],
    "Ток фазы А, A (ампер)": [0, 200],
    "Мощность активная, кВт": [0, 1000],
    "Напряжение, АВ Вольт": [0, 1000],
    "P затрубное, атм": [1, 300],
}

#Загрузка и объединение CSV
# Путь к папке с CSV-файлами
folder_path = tm_train

all_dfs = []
i = 0

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        path = os.path.join(folder_path, filename)
        df = pd.read_csv(path)

        # Название параметра по ID
        df["param_name"] = df["param_id"].map(id_to_param_name)

        # Pivot: long → wide
        df_pivot = df.pivot_table(
            index=["well_id", "tm_time"],
            columns="param_name",
            values="tm_value"
        ).reset_index()

        all_dfs.append(df_pivot)
        i += 1
        print(f"Обработан файл {i}: {filename}")

# Собираем все данные в один DataFrame
df_full = pd.concat(all_dfs, ignore_index=True).sort_values(["well_id", "tm_time"])

#Очистка по допустимым диапазонам
def filter_by_range(df, value_ranges):
    mask = pd.Series([True] * len(df), index=df.index)
    for col, (min_val, max_val) in value_ranges.items():
        if col in df.columns:
            # Оставляем, если значение либо в пределах, либо отсутствует
            col_mask = df[col].isna() | ((df[col] >= min_val) & (df[col] <= max_val))
            mask &= col_mask
    return df[mask]

df_full_cleaned = filter_by_range(df_full, value_ranges)




Обработан файл 1: train_10001.csv
Обработан файл 2: train_10002.csv
Обработан файл 3: train_10003.csv
Обработан файл 4: train_10004.csv
Обработан файл 5: train_10005.csv
Обработан файл 6: train_20001.csv
Обработан файл 7: train_20002.csv
Обработан файл 8: train_20003.csv
Обработан файл 9: train_20004.csv
Обработан файл 10: train_20005.csv
Обработан файл 11: train_20006.csv
Обработан файл 12: train_20007.csv
Обработан файл 13: train_20008.csv
Обработан файл 14: train_20009.csv
Обработан файл 15: train_20010.csv
Обработан файл 16: train_20011.csv
Обработан файл 17: train_20012.csv
Обработан файл 18: train_20013.csv
Обработан файл 19: train_20014.csv
Обработан файл 20: train_20015.csv
Обработан файл 21: train_20016.csv
Обработан файл 22: train_20017.csv
Обработан файл 23: train_20018.csv
Обработан файл 24: train_20019.csv
Обработан файл 25: train_20020.csv
Обработан файл 26: train_20021.csv
Обработан файл 27: train_20022.csv
Обработан файл 28: train_20023.csv
Обработан файл 29: train_2002

In [19]:
print(len(df_full))

17190717


In [ ]:
# Удаляем строки, где отсутствует значение дебита жидкости (target)
df_full_cleaned = df_full_cleaned.dropna(subset=["Дебит жидкости (объёмный), м3/сут"])
print(len(df_full_cleaned))

18533


In [21]:
df_full_cleaned.to_excel("df_full_cleaned.xlsx")

In [ ]:
# Загружаем предварительно очищенные данные из Excel-файлов
df_full_cleaned = pd.read_excel("df_full_cleaned.xlsx")
df_filtered = pd.read_excel("df_filtered.xlsx")

In [23]:
df_final = df_filtered

In [24]:
df_final.head(5)

,Unnamed: 0,ЭЦН_rate_nom_sm3day,Относительная плотность воды,ЭЦН_rate_opt_max_sm3day,НРХ_eff_area,ЭЦН_rate_opt_min_sm3day,ЭЦН_rate_max_sm3day,ЭЦН_eff_max,ПЭД_motor_nom_power,ПЭД_motor_nom_i,...,ЭК_MD_list_mean,Относительная плотность газа,ПЭД_motor_nom_eff,Штуцер_d,Пластовая температура,Линейная температура,ЭК_d_list_mean,Относительная плотность нефти,well_id,средний дебит
0,0,200,1.021,260,139.055892,161,325.937,0.6408,160.000000,59.0,...,1787.066667,0.7,0.8400,32.0,97.0,16.8802,147.933333,0.828,10001,223.828662
1,1,200,1.017,250,126.750000,150,300.000,0.6200,140.248269,57.0,...,2501.425000,0.7,0.8450,18.0,89.0,5.0000,146.850000,0.851,10002,178.558880
2,2,160,1.017,200,97.627250,125,245.000,0.6090,100.000000,30.2,...,2951.233333,0.7,0.8450,18.0,89.0,5.0000,142.600000,0.851,10003,108.583103
3,3,100,1.011,140,64.640000,70,160.000,0.5800,80.000000,30.7,...,2313.896667,0.7,0.8220,NaN,84.0,5.1522,139.466667,0.868,10004,60.179658
4,4,85,1.023,113,59.756550,53,156.000,0.5822,56.000000,35.3,...,1788.200000,0.7,0.8027,32.0,80.0,5.0000,117.200000,0.830,10005,1092.581341


In [25]:
df_full_cleaned.head(5)

,Unnamed: 0,well_id,tm_time,"P затрубное, атм","Давление линейное, атм","Давление на приеме насоса, атм","Дебит жидкости (объёмный), м3/сут","Загрузка ПЭД, %","Мощность активная, кВт","Напряжение, АВ Вольт","Обводненность (объёмная), %",Попутный газ,"Рбуф, атм","Ток фазы А, A (ампер)","Частота тока, Герц"
0,168,10001,2024-05-14 05:46:00,NaN,NaN,NaN,200.447,NaN,NaN,NaN,89.80,2103.82,NaN,NaN,NaN
1,348,10001,2024-05-14 11:46:00,NaN,NaN,NaN,178.901,NaN,NaN,NaN,90.70,1869.65,NaN,NaN,NaN
2,527,10001,2024-05-14 17:44:59,NaN,NaN,NaN,201.678,NaN,NaN,NaN,90.96,1988.58,NaN,NaN,NaN
3,705,10001,2024-05-14 23:46:00,NaN,NaN,NaN,198.947,NaN,NaN,NaN,89.05,2297.25,NaN,NaN,NaN
4,879,10001,2024-05-15 05:44:00,NaN,NaN,NaN,175.917,NaN,NaN,NaN,89.92,1975.90,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np

df_full_cleaned = pd.read_excel("df_full_cleaned.xlsx")
df_final = pd.read_excel("df_filtered.xlsx")
# Убедимся, что совпадают типы
df_final["well_id"] = df_final["well_id"].astype(str)
df_full_cleaned["well_id"] = df_full_cleaned["well_id"].astype(str)

# Объединяем по скважине
df_joined = df_full_cleaned.merge(
    df_final,
    on="well_id",
    how="left"
)
# Только числовые переменные
num_df = df_joined.select_dtypes(include='number')

# Целевая переменная
target = "Дебит жидкости (объёмный), м3/сут"

# Считаем корреляцию с целевой переменной
corr_with_target = num_df.corr()[target].sort_values(key=np.abs, ascending=False)
print(corr_with_target)
# Отбираем 6–7 наиболее коррелирующих признаков (исключая сам таргет)
top_corr_features = corr_with_target.drop(target).head(15).index.tolist()

print("🔝 Топ признаков по корреляции с дебитом:")
print(top_corr_features)

# Отфильтровываем датафрейм
df_selected = df_joined[top_corr_features + [target]]
print("\n✅ Итоговый датафрейм с выбранными признаками:")
print(df_selected.head())

print("📌 Объединённый датафрейм:")
print(df_joined.head())

Дебит жидкости (объёмный), м3/сут    1.000000
Мощность активная, кВт               0.933391
Ток фазы А, A (ампер)                0.917285
ЭЦН_rate_nom_sm3day                  0.773733
ЭЦН_rate_opt_min_sm3day              0.771597
ЭЦН_rate_opt_max_sm3day              0.768042
НРХ_eff_area                         0.756551
ЭЦН_rate_max_sm3day                  0.736997
ПЭД_motor_nom_power                  0.732810
НРХ_head_area                        0.652817
НРХ_power_area                       0.634623
ЭЦН_eff_max                          0.621261
ПЭД_motor_amp_idle                   0.588537
ПЭД_motor_nom_i                      0.584924
Напряжение, АВ Вольт                -0.542492
средний дебит                        0.540100
Unnamed: 0_y                        -0.516564
Загрузка ПЭД, %                      0.500583
Давление на приеме насоса, атм       0.467644
Штуцер_d                            -0.442562
Относительная плотность воды         0.381548
ПЭД_motor_nom_voltage             

In [29]:
!pip install -U lightgbm
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00


In [37]:
import pandas as pd
import numpy as np
import datetime
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Целевая переменная
target_col = "Дебит жидкости (объёмный), м3/сут"

# Удаляем лишние столбцы
drop_cols = ["Относительная плотность газа", "well_id", "ID скважины", "средний дебит", target_col]

# Формируем X и y
X = df_selected.drop(columns=drop_cols, errors="ignore")
y = df_selected[target_col]

# Преобразуем datetime.date и pd.Timestamp в строки для обработки CatBoost
for col in X.columns:
    if X[col].apply(lambda x: isinstance(x, (datetime.date, pd.Timestamp))).any():
        print(f"Преобразуем {col} в строку для CatBoost")
        X[col] = X[col].astype(str)

# Определяем категориальные признаки
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
X = X.loc[:, ~X.columns.str.startswith("Unnamed")]

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Создаем пулы
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

# Модель CatBoost
model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.01,
    depth=6,
    loss_function='RMSE',
    eval_metric='MAPE',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=100
)

# Обучение
print("Начало обучения CatBoost...")
model.fit(train_pool, eval_set=test_pool)

# Предсказание и оценка
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"\nRMSE на тестовой выборке: {rmse:.3f}")

# Важность признаков
feat_imp = pd.DataFrame({
    'feature': model.feature_names_,
    'importance': model.get_feature_importance()
}).sort_values(by='importance', ascending=False)

print("\nТоп-10 важных признаков:")
print(feat_imp.head(10))


Начало обучения CatBoost...
0:	learn: 6.7939808	test: 7.4756274	best: 7.4756274 (0)	total: 4.4ms	remaining: 44s
100:	learn: 4.7614790	test: 5.3166682	best: 5.3166682 (100)	total: 338ms	remaining: 33.1s
200:	learn: 3.8313421	test: 4.3319123	best: 4.3319123 (200)	total: 662ms	remaining: 32.3s
300:	learn: 3.3461511	test: 3.8128200	best: 3.8128200 (300)	total: 1.01s	remaining: 32.7s
400:	learn: 3.0421111	test: 3.4891638	best: 3.4891638 (400)	total: 1.34s	remaining: 32.1s
500:	learn: 2.8297315	test: 3.2645553	best: 3.2645553 (500)	total: 1.66s	remaining: 31.5s
600:	learn: 2.6858305	test: 3.1146463	best: 3.1146463 (600)	total: 1.98s	remaining: 30.9s
700:	learn: 2.5983633	test: 3.0226948	best: 3.0226948 (700)	total: 2.3s	remaining: 30.6s
800:	learn: 2.5201774	test: 2.9414282	best: 2.9414282 (800)	total: 2.61s	remaining: 30s
900:	learn: 2.4637325	test: 2.8828325	best: 2.8828325 (900)	total: 2.92s	remaining: 29.5s
1000:	learn: 2.4228392	test: 2.8405694	best: 2.8405694 (1000)	total: 3.27s	remain

In [ ]:
# Функция для безопасного расчёта MAPE (процент ошибки), с заменой нулей в y_true на малое число
def safe_mape(y_true, y_pred):
    y_true_safe = np.where(y_true == 0, 1e-6, y_true)  # заменим нули
    return np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

# Расчёт MAPE по тестовой выборке
mape = safe_mape(y_test.values, y_pred)
print(f"MAPE на тестовой выборке: {mape:.2f}%")



MAPE на тестовой выборке: 157569132.49%


In [39]:
model.save_model("catboost_model.cbm")

In [40]:
print(model.feature_names_)

['Мощность активная, кВт', 'Ток фазы А, A (ампер)', 'ЭЦН_rate_nom_sm3day', 'ЭЦН_rate_opt_min_sm3day', 'ЭЦН_rate_opt_max_sm3day', 'НРХ_eff_area', 'ЭЦН_rate_max_sm3day', 'ПЭД_motor_nom_power', 'НРХ_head_area', 'НРХ_power_area', 'ЭЦН_eff_max', 'ПЭД_motor_amp_idle', 'ПЭД_motor_nom_i', 'Напряжение, АВ Вольт']


In [41]:
PATH = 'drive/MyDrive/day1'
test_path = f'{PATH}/test_dataset_1'
st_test_data = f'{test_path}/static_data/train_test_static_data.xlsx'
tm_test = f'{test_path}/tm_data'

In [62]:
df = pd.read_excel(st_test_data)

In [63]:
import pandas as pd
import json
import ast
# Безопасная функция для разбора строковых словарей
def safe_load(val):
    if isinstance(val, str):
        try:
            return json.loads(val)
        except:
            return ast.literal_eval(val)
    return val

# Разворачиваем словарь, но оставляем списки как есть
def expand_dict_column_keep_lists(df, column_name, prefix):
    df[column_name] = df[column_name].apply(safe_load)

    def process_dict(d):
        if not isinstance(d, dict):
            return pd.Series({})  # Возвращаем пустую строку
        return pd.Series({f"{prefix}_{k}": v for k, v in d.items()})

    return df[column_name].apply(process_dict)


# Разворачиваем sections_info с ограничением по числу секций
def expand_section_info_as_lists(column_data, prefix):
    expanded = []
    for row in column_data:
        row = safe_load(row)
        sections = row.get("sections_info", [])
        md_list = []
        d_list = []
        for sec in sections:
            md_list.append(sec.get("MD"))
            d_list.append(sec.get("d"))
        expanded.append({
            f"{prefix}_bottom_depth": row.get("bottom_depth"),
            f"{prefix}_roughness": row.get("roughness", None),
            f"{prefix}_MD_list": md_list,
            f"{prefix}_d_list": d_list
        })
    return pd.DataFrame(expanded)


# Копия исходного датафрейма
df_exp = df.copy()

# Разворачиваем нужные словарные поля
df_ecn = expand_dict_column_keep_lists(df_exp, "Паспортные данные ЭЦН", "ЭЦН")
df_ped = expand_dict_column_keep_lists(df_exp, "Характеристики ПЭД", "ПЭД")
df_sep = expand_dict_column_keep_lists(df_exp, "Сепаратор", "Сепаратор")
df_sht = expand_dict_column_keep_lists(df_exp, "Штуцер", "Штуцер")

# Обрабатываем инклинометрию — только первые значения для наглядности
df_incl = df_exp["Инклинометрия"].apply(safe_load)
df_exp["Инклинометрия_measured_0"] = df_incl.apply(lambda x: x["measured"][0] if "measured" in x and x["measured"] else None)
df_exp["Инклинометрия_absolute_0"] = df_incl.apply(lambda x: x["absolute"][0] if "absolute" in x and x["absolute"] else None)

# ЭК и НКТ
df_ek = expand_section_info_as_lists(df_exp["ЭК"], "ЭК")
df_nkt = expand_section_info_as_lists(df_exp["НКТ"], "НКТ")


# Финальный датафрейм — объединяем всё
df_flat = pd.concat([
    df_exp.drop(columns=[
        "Паспортные данные ЭЦН", "Характеристики ПЭД", "Сепаратор", "Штуцер",
        "Инклинометрия", "ЭК", "НКТ"
    ]),
    df_ecn,
    df_ped,
    df_sep,
    df_sht,
    df_ek,
    df_nkt
], axis=1)


In [ ]:
from numpy import trapz  

def nrx_area_features(df, x_col, y_cols):
    area_data = {}

    for y in y_cols:
        base = y.replace("ЭЦН_", "").replace("_points", "")  # "ЭЦН_head_points" -> "head"
        area_data[f"НРХ_{base}_area"] = []

    for _, row in df.iterrows():
        x = row.get(x_col)

        for y in y_cols:
            base = y.replace("ЭЦН_", "").replace("_points", "")
            y_vals = row.get(y)

            if isinstance(x, list) and isinstance(y_vals, list) and len(x) == len(y_vals) and len(x) >= 2:
                area = trapz(y_vals, x)  # площадь под кривой
            else:
                area = np.nan

            area_data[f"НРХ_{base}_area"].append(area)

    return pd.DataFrame(area_data)
df_area = nrx_area_features(
    df=df_flat,
    x_col="ЭЦН_rate_points",
    y_cols=["ЭЦН_head_points", "ЭЦН_power_points", "ЭЦН_eff_points"]
)

# Объединяем с основным датафреймом
df_flat = pd.concat([df_flat.drop(columns=["ЭЦН_rate_points"]), df_area], axis=1)

<ipython-input-64-2eecad0bf7fe>:18: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = trapz(y_vals, x)  # площадь под кривой


In [65]:
from numpy import trapz

def motor_area_features(df, x_col, y_cols):
    area_data = {}

    for y in y_cols:
        base = y.replace("ПЭД_", "").replace("_points", "")
        area_data[f"ПЭД_{base}_area"] = []

    for _, row in df.iterrows():
        x = row.get(x_col)

        for y in y_cols:
            base = y.replace("ПЭД_", "").replace("_points", "")
            y_vals = row.get(y)

            if isinstance(x, list) and isinstance(y_vals, list) and len(x) == len(y_vals) and len(x) >= 2:
                area = trapz(y_vals, x)
            else:
                area = np.nan

            area_data[f"ПЭД_{base}_area"].append(area)

    return pd.DataFrame(area_data)
df_motor_area = motor_area_features(
    df=df_flat,
    x_col="ПЭД_load_points",
    y_cols=[
        "ПЭД_amperage_points",
        "ПЭД_cosf_points",
        "ПЭД_eff_points",
        "ПЭД_rpm_points"
    ]
)

# Объединяем с исходным DataFrame и удаляем ПЭД_load_points
df_flat = pd.concat([df_flat.drop(columns=["ПЭД_load_points"]), df_motor_area], axis=1)

# Смотрим результат
print(df_flat.filter(like="ПЭД_").head())


  ПЭД_manufacturer                ПЭД_name  ПЭД_d_motor_mm  ПЭД_motor_nom_i  \
0           СЦ ЭПУ          ПЭДМТ160-117М1           117.0             59.0   
1   Лысьванефтемаш          1ЭДБ140-117 В5           117.0             57.0   
2            Борец  9ЭДБТ100-117Э/2700М8В5           117.0             30.2   
3          Новомет     ПЭДН80-117-2100/041           117.0             30.7   
4     Schlumberger             ПЭДМТ56-103           103.0             35.3   

   ПЭД_motor_amp_idle  ПЭД_motor_nom_power  ПЭД_motor_nom_voltage  \
0                59.0           160.000000                 2250.0   
1                57.0           140.248269                 2000.0   
2                30.2           100.000000                 2700.0   
3                30.7            80.000000                 2100.0   
4                35.3            56.000000                 1400.0   

   ПЭД_motor_nom_eff  ПЭД_motor_nom_cosf  ПЭД_motor_nom_freq  \
0             0.8400               0.840      

<ipython-input-65-c5ba41b69c5e>:18: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = trapz(y_vals, x)


In [ ]:
cols_to_avg = ["ЭК_MD_list", "ЭК_d_list", "НКТ_MD_list", "НКТ_d_list"]

for col in cols_to_avg:
    df_flat[f"{col}_mean"] = df_flat[col].apply(
        lambda x: sum(x)/len(x) if isinstance(x, list) and len(x) > 0 else np.nan
    )

# Теперь можем удалить оригинальные массивы
df_flat.drop(columns=cols_to_avg, inplace=True)

print(df_flat[[col + "_mean" for col in cols_to_avg]].head())


   ЭК_MD_list_mean  ЭК_d_list_mean  НКТ_MD_list_mean  НКТ_d_list_mean
0      1787.066667      147.933333       1924.275000            59.00
1      2501.425000      146.850000       2034.012500            59.00
2      2951.233333      142.600000       1837.572500            59.00
3      2313.896667      139.466667       1752.256667            58.00
4      1788.200000      117.200000       1540.072500            58.75


In [67]:
required_columns = ['ID скважины', 'well_id', 'Мощность активная, кВт', 'Ток фазы А, A (ампер)', 'ЭЦН_rate_nom_sm3day', 'ЭЦН_rate_opt_min_sm3day', 'ЭЦН_rate_opt_max_sm3day', 'НРХ_eff_area', 'ЭЦН_rate_max_sm3day', 'ПЭД_motor_nom_power', 'НРХ_head_area', 'НРХ_power_area', 'ЭЦН_eff_max', 'ПЭД_motor_amp_idle', 'ПЭД_motor_nom_i', 'Напряжение, АВ Вольт']

# Отфильтровываем только те колонки, которые реально есть в df_flat
filtered_columns = [col for col in required_columns if col in df_flat.columns]
df_flat_filtered = df_flat[filtered_columns]

In [ ]:
import pandas as pd
import os

# Названия по param_id
id_to_param_name = {
    1: "Дебит жидкости (объёмный), м3/сут",
    2: "Обводненность (объёмная), %",
    3: "Рбуф, атм",
    4: "Давление линейное, атм",
    5: "Давление на приеме насоса, атм",
    6: "Загрузка ПЭД, %",
    7: "Попутный газ",
    8: "Частота тока, Герц",
    9: "Ток фазы А, A (ампер)",
    10: "Мощность активная, кВт",
    11: "Напряжение, АВ Вольт",
    12: "P затрубное, атм",
}

# Диапазоны допустимых значений
value_ranges = {
    "Дебит жидкости (объёмный), м3/сут": [0, 5000],
    "Обводненность (объёмная), %": [0, 100],
    "Рбуф, атм": [1, 300],
    "Давление линейное, атм": [1, 300],
    "Давление на приеме насоса, атм": [1, 300],
    "Загрузка ПЭД, %": [0, 100],
    "Попутный газ": [0, 100000],
    "Частота тока, Герц": [0, 500],
    "Ток фазы А, A (ампер)": [0, 200],
    "Мощность активная, кВт": [0, 1000],
    "Напряжение, АВ Вольт": [0, 1000],
    "P затрубное, атм": [1, 300],
}

#Загрузка и объединение CSV 
folder_path = tm_test

all_dfs = []
i = 0

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        path = os.path.join(folder_path, filename)
        df = pd.read_csv(path)

        # Название параметра по ID
        df["param_name"] = df["param_id"].map(id_to_param_name)

        # Pivot: long → wide
        df_pivot = df.pivot_table(
            index=["well_id", "tm_time"],
            columns="param_name",
            values="tm_value"
        ).reset_index()

        all_dfs.append(df_pivot)
        i += 1
        print(f"Обработан файл {i}: {filename}")

# Собираем все данные в один DataFrame
df_full = pd.concat(all_dfs, ignore_index=True).sort_values(["well_id", "tm_time"])
print(len(df_full))

# Очистка по допустимым диапазонам
def filter_by_range(df, value_ranges):
    mask = pd.Series([True] * len(df), index=df.index)
    for col, (min_val, max_val) in value_ranges.items():
        if col in df.columns:
            # Оставляем, если значение либо в пределах, либо отсутствует
            col_mask = df[col].isna() | ((df[col] >= min_val) & (df[col] <= max_val))
            mask &= col_mask
    return df[mask]

df_full_cleaned = filter_by_range(df_full, value_ranges)
print(len(df_full_cleaned))

Обработан файл 1: test_10001.csv
Обработан файл 2: test_10002.csv
Обработан файл 3: test_10003.csv
Обработан файл 4: test_10004.csv
Обработан файл 5: test_10005.csv
Обработан файл 6: test_20001.csv
Обработан файл 7: test_20002.csv
Обработан файл 8: test_20003.csv
Обработан файл 9: test_20004.csv
Обработан файл 10: test_20005.csv
Обработан файл 11: test_20006.csv
Обработан файл 12: test_20007.csv
Обработан файл 13: test_20008.csv
Обработан файл 14: test_20009.csv
Обработан файл 15: test_20010.csv
Обработан файл 16: test_20011.csv
Обработан файл 17: test_20012.csv
Обработан файл 18: test_20013.csv
Обработан файл 19: test_20014.csv
Обработан файл 20: test_20015.csv
Обработан файл 21: test_20016.csv
Обработан файл 22: test_20017.csv
Обработан файл 23: test_20018.csv
Обработан файл 24: test_20019.csv
Обработан файл 25: test_20020.csv
Обработан файл 26: test_20021.csv
Обработан файл 27: test_20022.csv
Обработан файл 28: test_20023.csv
Обработан файл 29: test_20024.csv
Обработан файл 30: test

In [68]:
print("df_flat_filtered columns:", df_flat_filtered.columns.tolist())
print("df_full_cleaned columns:", df_full_cleaned.columns.tolist())


df_flat_filtered columns: ['ID скважины', 'ЭЦН_rate_nom_sm3day', 'ЭЦН_rate_opt_min_sm3day', 'ЭЦН_rate_opt_max_sm3day', 'НРХ_eff_area', 'ЭЦН_rate_max_sm3day', 'ПЭД_motor_nom_power', 'НРХ_head_area', 'НРХ_power_area', 'ЭЦН_eff_max', 'ПЭД_motor_amp_idle', 'ПЭД_motor_nom_i']
df_full_cleaned columns: ['well_id', 'tm_time', 'Давление линейное, атм', 'Давление на приеме насоса, атм', 'Дебит жидкости (объёмный), м3/сут', 'Загрузка ПЭД, %', 'Мощность активная, кВт', 'Напряжение, АВ Вольт', 'Обводненность (объёмная), %', 'Попутный газ', 'Ток фазы А, A (ампер)', 'Частота тока, Герц', 'P затрубное, атм', 'Рбуф, атм']


In [69]:
df_flat_filtered = df_flat_filtered.rename(columns={"ID скважины": "well_id"})
print(df_flat_filtered.head())
print(df_full_cleaned.head())

df_flat_filtered["well_id"] = df_flat_filtered["well_id"].astype(str)
df_full_cleaned["well_id"] = df_full_cleaned["well_id"].astype(str)

# Объединяем по скважине
df_joined = df_full_cleaned.merge(
    df_flat_filtered,
    on="well_id",
    how="left"
)

print("📌 Объединённый датафрейм:")
print(df_joined.head())

   well_id  ЭЦН_rate_nom_sm3day  ЭЦН_rate_opt_min_sm3day  \
0    10001                200.0                    161.0   
1    10002                200.0                    150.0   
2    10003                160.0                    125.0   
3    10004                100.0                     70.0   
4    10005                 85.0                     53.0   

   ЭЦН_rate_opt_max_sm3day  НРХ_eff_area  ЭЦН_rate_max_sm3day  \
0                    260.0    139.055892              325.937   
1                    250.0    126.750000              300.000   
2                    200.0     97.627250              245.000   
3                    140.0     64.640000              160.000   
4                    113.0     59.756550              156.000   

   ПЭД_motor_nom_power  НРХ_head_area  НРХ_power_area  ЭЦН_eff_max  \
0           160.000000    1800.417551       64.394247       0.6408   
1           140.248269    1488.500000       47.752500       0.6200   
2           100.000000    1299.750000 

<ipython-input-69-634a98c97d15>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_cleaned["well_id"] = df_full_cleaned["well_id"].astype(str)


📌 Объединённый датафрейм:
  well_id              tm_time  Давление линейное, атм  \
0   10001  2024-10-14 00:00:41                     NaN   
1   10001  2024-10-14 00:02:41                     NaN   
2   10001  2024-10-14 00:04:41                     NaN   
3   10001  2024-10-14 00:06:42                     NaN   
4   10001  2024-10-14 00:09:11                     NaN   

   Давление на приеме насоса, атм  Дебит жидкости (объёмный), м3/сут  \
0                            55.8                                NaN   
1                            55.8                                NaN   
2                            55.8                                NaN   
3                            55.8                                NaN   
4                            55.8                                NaN   

   Загрузка ПЭД, %  Мощность активная, кВт  Напряжение, АВ Вольт  \
0             78.0                   204.0                 399.0   
1             78.0                   204.0              

In [82]:
# Целевая переменная
target_col = "Дебит жидкости (объёмный), м3/сут"

# Удаляем строки с отсутствующим значением target
df_predict = df_joined.dropna(subset=[target_col]).copy()

# Отбрасываем ненужные столбцы
drop_cols = ["Относительная плотность газа", "well_id", "ID скважины", "средний дебит", target_col]
X_test_real = df_predict.drop(columns=drop_cols, errors="ignore").copy()

# Преобразуем datetime в строки
for col in X_test_real.columns:
    if X_test_real[col].apply(lambda x: isinstance(x, (datetime.date, pd.Timestamp))).any():
        X_test_real[col] = X_test_real[col].astype(str)

# Убедимся, что колонки совпадают с обучающей моделью
X_test_real = X_test_real[model.feature_names_]

# Обновляем целевую переменную
y_test_true = df_predict[target_col].values

# Предсказания
y_test_pred = model.predict(X_test_real)

print("✅ Предсказания выполнены успешно.")


✅ Предсказания выполнены успешно.


In [83]:
print(len(X_test_real))

3617


In [ ]:
# Расчёт MAPE
def safe_mape(y_true, y_pred):
    y_true_safe = np.where(y_true == 0, 1e-6, y_true)  # избегаем деления на 0
    return np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

def safe_mape(y_true, y_pred):
    y_true_safe = np.where(y_true == 0, 1e-6, y_true)  # заменим нули
    return np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

mape = safe_mape(y_test.values, y_pred)
print(f"MAPE на тестовой выборке: {mape:.2f}%")

MAPE на тестовой выборке: 157569132.49%


In [76]:
import numpy as np
import pandas as pd

# === Объединяем всё в DataFrame для оценки ===
df_eval = pd.DataFrame({
    "well_id": df_joined["well_id"].values,
    "y_true": y_test_true,
    "y_pred": y_test_pred
})

# === Базовая модель: предсказывает медиану по всей выборке ===
median_baseline = np.median(y_test_true)
df_eval["y_baseline"] = median_baseline

# === Считаем средние значения по каждой скважине ===
df_avg = df_eval.groupby("well_id").mean(numeric_only=True).reset_index()

# === Функция расчёта MAPE без нулей ===
def safe_mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# === Расчёт MAPE ===
mape_model = safe_mape(df_avg["y_true"], df_avg["y_pred"])
mape_baseline = safe_mape(df_avg["y_true"], df_avg["y_baseline"])

# === Вывод результатов ===
print(f"\n📊 MAPE базовой (медианной) модели по скважинам: {mape_baseline:.2f}%")
print(f"🎯 MAPE модели CatBoost по скважинам: {mape_model:.2f}%")
print(f"💪 Улучшение: {mape_baseline - mape_model:.2f} процентных пункта")



📊 MAPE базовой (медианной) модели по скважинам: nan%
🎯 MAPE модели CatBoost по скважинам: 47.59%
💪 Улучшение: nan процентных пункта


In [77]:
for col in model.feature_names_:
    print(f"{col}: {X[col].dtype}")


Мощность активная, кВт: float64
Ток фазы А, A (ампер): float64
ЭЦН_rate_nom_sm3day: int64
ЭЦН_rate_opt_min_sm3day: int64
ЭЦН_rate_opt_max_sm3day: int64
НРХ_eff_area: float64
ЭЦН_rate_max_sm3day: float64
ПЭД_motor_nom_power: float64
НРХ_head_area: float64
НРХ_power_area: float64
ЭЦН_eff_max: float64
ПЭД_motor_amp_idle: float64
ПЭД_motor_nom_i: float64
Напряжение, АВ Вольт: float64


In [79]:
sample_input = {
    "Мощность активная, кВт": 180.5,
    "Ток фазы А, A (ампер)": 85.3,
    "ЭЦН_rate_nom_sm3day": 220,
    "ЭЦН_rate_opt_min_sm3day": 160,
    "ЭЦН_rate_opt_max_sm3day": 260,
    "НРХ_eff_area": 110.2,
    "ЭЦН_rate_max_sm3day": 300.0,
    "ПЭД_motor_nom_power": 160.0,
    "НРХ_head_area": 145.0,
    "НРХ_power_area": 175.5,
    "ЭЦН_eff_max": 0.62,
    "ПЭД_motor_amp_idle": 26.7,
    "ПЭД_motor_nom_i": 56.3,
    "Напряжение, АВ Вольт": 380.0
}

# Создаём DataFrame из одного наблюдения
X_test_sample = pd.DataFrame([sample_input])


# Предсказание
y_pred_sample = model.predict(X_test_sample)

print(f"\n🧠 Предсказанный дебит: {y_pred_sample[0]:.2f} м³/сут")


🧠 Предсказанный дебит: 142.15 м³/сут
